In [9]:
from infra.utils import construct_cluster_names, scale_cluster, list_tpus, check_tpu, validate_cluster, get_pipelines
from infra.config import  cluster_config, constant_args
from infra.sentry import connect, run, clear_all, LEADER, tmux, tmux_shell

from tqdm import tqdm

In [2]:

# Our primary maintenence loop needs to do the following:

# Get a list of TPUs

# If a  TPU is active and has a spot in the pipelines then do nothing

# If a TPU is not active, and it is in a pipeline, delete the TPU and halt all training loops in that pipeline.
# (It was pre-empted and the others will be hanging waiting for it) Maybe we can set it up so that if a training loop is interrupted, it catches the 
cluster_config

{'nodes': 4,
 'pipeline_length': 2,
 'name': 'test',
 'project': 'learning-from-play-303306',
 'accelerator_type': 'v2-8',
 'zone': 'us-central1-f',
 'preemptible': False,
 'redis_password': '5241590000000000'}

In [20]:
operating_tpus[0]

{'name': 'projects/learning-from-play-303306/locations/us-central1-f/nodes/test-2',
 'acceleratorType': 'v2-8',
 'state': 'READY',
 'runtimeVersion': 'v2-alpha',
 'cidrBlock': '10.128.0.0/20',
 'createTime': '2022-02-21T02:18:22.855697646Z',
 'schedulingConfig': {},
 'networkEndpoints': [{'ipAddress': '10.128.15.204',
   'port': 8470,
   'accessConfig': {'externalIp': '35.239.96.195'}}],
 'health': 'HEALTHY',
 'id': '4796306648445191617',
 'networkConfig': {'network': 'projects/learning-from-play-303306/global/networks/default',
  'subnetwork': 'projects/learning-from-play-303306/regions/us-central1/subnetworks/default',
  'enableExternalIps': True},
 'serviceAccount': {'email': '940527310713-compute@developer.gserviceaccount.com',
  'scope': ['https://www.googleapis.com/auth/devstorage.read_write',
   'https://www.googleapis.com/auth/logging.write',
   'https://www.googleapis.com/auth/service.management',
   'https://www.googleapis.com/auth/servicecontrol',
   'https://www.googleapis.

In [4]:
scale_cluster()

In [5]:
# the next thing we need to do is look at avilable to make and create config


valid_pipelines = get_pipelines()
operating_tpus =  [tpu for pipeline in valid_pipelines for tpu in pipeline]

In [25]:
connect()


In [27]:
run('cd scalingExperiments/pipelineParallelism')

In [25]:
run('git clone https://github.com/sholtodouglas/scalingExperiments')
run('cd scalingExperiments/pipelineParallelism')
run('pip install -U "ray[default]"')
# add the ray scripts to path - they are not accessible by default
run('echo "export PATH="$HOME/.local/bin:$PATH"" >> ~/.bashrc')
run('source ~/.bashrc')

In [32]:
run('sudo apt-get update')
run('sudo apt install -y libopenmpi-dev')
run('pip install mpi4jax') # Todo this often doesn't get called .. I think b/c of the y?

In [17]:
def copy_file(path, dest):
    for tpu in tqdm(operating_tpus):
        tpu['connection_object'].put(path, dest)



In [73]:
import subprocess

command = subprocess.run("ssh-keygen -t rsa -f id_rsa -N '' -C 'MPI Keys'", shell=True)

for tpu in tqdm(operating_tpus):
    # Remove the strict host checking requirement
    tpu['connection_object'].run('echo "StrictHostKeyChecking no" | sudo tee --append /etc/ssh/ssh_config')
    # give every node the same ssh keys
    tpu['connection_object'].put('id_rsa', '.ssh')
    tpu['connection_object'].put('id_rsa.pub', '.ssh')
    # put the public key in authorised keys for every node
    tpu['connection_object'].run('cat .ssh/id_rsa.pub >> .ssh/authorized_keys')

  0%|          | 0/4 [00:00<?, ?it/s]

StrictHostKeyChecking no


 25%|██▌       | 1/4 [00:03<00:11,  3.83s/it]

StrictHostKeyChecking no


 50%|█████     | 2/4 [00:07<00:07,  3.81s/it]

StrictHostKeyChecking no


 75%|███████▌  | 3/4 [00:11<00:03,  3.80s/it]

StrictHostKeyChecking no


100%|██████████| 4/4 [00:15<00:00,  3.79s/it]


In [74]:
with open('infra/host_list', 'w') as f:
    for idx, tpu  in enumerate(operating_tpus):
        # f.write(+'\n')
        f.write(tpu['networkEndpoints'][0]['ipAddress']+ ' slots=1\n')

copy_file('infra/host_list', 'scalingExperiments/pipelineParallelism/infra')    


In [30]:
copy_file('infra/mpi_test.py', 'scalingExperiments/pipelineParallelism/infra')

100%|██████████| 4/4 [00:05<00:00,  1.39s/it]


In [78]:
run('mpirun --hostfile infra/host_list python3 infra/mpi_test.py',0)

In [91]:
run('^C')

In [ ]:
#############################################################################################################################
run('ray start --head --port=6379', LEADER)

In [ ]:
# set up ray on all the follower nodes
for idx, tpu  in enumerate(operating_tpus):
    if idx != LEADER:
        run(f"ray start --address='{operating_tpus[LEADER]['networkEndpoints'][0]['ipAddress']}:6379' \
                        --redis-password='{cluster_config['redis_password']}'", \
                        idx)

In [ ]:
run('python3 infra/ray_test.py')

In [78]:
constant_args

'--zone=us-central1-f --project=learning-from-play-303306'

run('ray stop')

In [ ]:
10.128.15.204, 10.128.15.207, 10.128.15.201, 10.128.15.203

In [ ]:
mpirun -np 1 --hostfile infra/host_list --host t1v-n-e23727e8-w-0 python3 infra/mpi_test.py

In [88]:
run('ray stop')

In [ ]:
run('pwd')

In [ ]:
gcloud alpha compute scp infra/ray_test.py test-0:~/$HOME/

{'error': {'code': 401,
  'message': 'Request had invalid authentication credentials. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project.',
  'status': 'UNAUTHENTICATED',
  'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo',
    'reason': 'ACCESS_TOKEN_TYPE_UNSUPPORTED',
    'metadata': {'method': 'google.cloud.tpu.v2alpha1.Tpu.ListNodes',
     'service': 'tpu.googleapis.com'}}]}}

In [ ]:
validate_cluster(cluster_config['nodes'])

  0%|          | 0/6 [00:00<?, ?it/s]

Validating cluster creation


 17%|█▋        | 1/6 [00:01<00:08,  1.78s/it]

TPU test_0 not found


 33%|███▎      | 2/6 [00:03<00:06,  1.63s/it]

TPU test_1 not found


 83%|████████▎ | 5/6 [00:08<00:01,  1.69s/it]

TPU test_4 not found


100%|██████████| 6/6 [00:14<00:00,  2.38s/it]


In [ ]:
tmux_shell(f"gcloud alpha compute tpus tpu-vm ssh {'test2'} {constant_args} -- -L {8802}:localhost:{8802}")

In [ ]:
tmux("set -g pane-border-style bg=red fg=red")

In [ ]:
connect()

In [ ]:
import subprocess

command = subprocess.run(f"netstat - ant | grep {8805}",
    capture_output=True,  # Capture the output (can be accessed via the .stdout member)
    text=True,  # Capture output as text, not as bytes
    shell=True)  # Run in shell. Required, because you use pipes.
    
device_port = command.stdout  # Get the output and convert it to an int
print(device_port)  # Print the parsed port number

In [ ]:
/home/sholto/.local│  WARNING: The scripts ray, ray-operator, rllib, serve and tune are installed in '/home/sholto/.local/
/bin

[sconn(fd=40, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_STREAM: 1>, laddr=addr(ip='127.0.0.1', port=43975), raddr=addr(ip='127.0.0.1', port=50414), status='ESTABLISHED', pid=13912),
 sconn(fd=23, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_STREAM: 1>, laddr=addr(ip='127.0.0.1', port=44969), raddr=(), status='LISTEN', pid=13912),
 sconn(fd=8, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_STREAM: 1>, laddr=addr(ip='127.0.0.1', port=3080), raddr=(), status='LISTEN', pid=2496),
 sconn(fd=131, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_STREAM: 1>, laddr=addr(ip='127.0.0.1', port=45942), raddr=addr(ip='127.0.0.1', port=8888), status='ESTABLISHED', pid=1071),
 sconn(fd=7, family=<AddressFamily.AF_INET: 2>, type=<SocketKind.SOCK_STREAM: 1>, laddr=addr(ip='127.0.0.1', port=3085), raddr=(), status='LISTEN', pid=2496),
 sconn(fd=18, family=<AddressFamily.AF_INET6: 10>, type=<SocketKind.SOCK_STREAM: 1>, laddr=addr(ip='::', port=7000), raddr=(), 

In [ ]:

ports

[43975,
 44969,
 3080,
 45942,
 3085,
 7000,
 8888,
 45826,
 48506,
 5353,
 36655,
 45690,
 54706,
 40594,
 39853,
 48355,
 56992,
 49336,
 5353,
 8892,
 55647,
 8888,
 50414,
 59176,
 46406,
 46160,
 55619,
 631,
 48030,
 48286,
 8805,
 5353,
 53,
 48707,
 54704,
 52966,
 8888,
 43975,
 53,
 39432,
 39467,
 8888,
 59235,
 33198,
 3333,
 5353,
 60801,
 45589,
 68,
 57724,
 36203,
 8888,
 33190,
 55619,
 36655,
 33562,
 57408,
 45698,
 47344,
 8888,
 56662,
 58866,
 8800,
 56022,
 8805,
 41590,
 8888,
 631,
 8888,
 40116,
 39242,
 41584,
 8888,
 36636,
 41092,
 38478,
 3333,
 48824,
 8891,
 50280,
 8888,
 39853,
 45730,
 56660,
 8800,
 36655,
 52467,
 631,
 33010,
 3333,
 56020,
 46410,
 39434,
 8888,
 58198,
 54368,
 8888,
 8888,
 8888,
 8888,
 58668,
 34056,
 47328,
 33906,
 39853]

In [ ]:
make_tmux(6)

In [ ]:
scale_cluster(6)

Deleting test_1
Starting test_1
Deleting test_3
Starting test_3
Creating test_4
{'name': 'projects/learning-from-play-303306/locations/us-central1-f/operations/operation-1643935665172-5d726995b2f90-f51e4767-e303dfb7', 'metadata': {'@type': 'type.googleapis.com/google.cloud.common.OperationMetadata', 'createTime': '2022-02-04T00:47:46.994482809Z', 'target': 'projects/learning-from-play-303306/locations/us-central1-f/nodes/test_4', 'verb': 'create', 'cancelRequested': False, 'apiVersion': 'v2alpha1'}, 'done': False}
Creating test_5
{'name': 'projects/learning-from-play-303306/locations/us-central1-f/operations/operation-1643935669267-5d7269999ace0-c9e483cb-0308fdfa', 'metadata': {'@type': 'type.googleapis.com/google.cloud.common.OperationMetadata', 'createTime': '2022-02-04T00:47:50.986263737Z', 'target': 'projects/learning-from-play-303306/locations/us-central1-f/nodes/test_5', 'verb': 'create', 'cancelRequested': False, 'apiVersion': 'v2alpha1'}, 'done': False}


In [ ]:
check_tpu('test_1')

{'error': {'code': 404,
  'message': "Resource 'projects/learning-from-play-303306/locations/us-central1-f/nodes/test_1' was not found",
  'status': 'NOT_FOUND',
  'details': [{'@type': 'type.googleapis.com/google.rpc.ResourceInfo',
    'resourceName': 'projects/learning-from-play-303306/locations/us-central1-f/nodes/test_1'}]}}

In [ ]:
scale_cluster(4)

Creating test_1
{'name': 'projects/learning-from-play-303306/locations/us-central1-f/operations/operation-1643889519283-5d71bdad8cecc-cf195c1f-86a70427', 'metadata': {'@type': 'type.googleapis.com/google.cloud.common.OperationMetadata', 'createTime': '2022-02-03T11:58:41.098872920Z', 'target': 'projects/learning-from-play-303306/locations/us-central1-f/nodes/test_1', 'verb': 'create', 'cancelRequested': False, 'apiVersion': 'v2alpha1'}, 'done': False}


In [ ]:
validate_cluster(6)

  0%|          | 0/6 [00:00<?, ?it/s]

Validating cluster creation


 33%|███▎      | 2/6 [00:04<00:09,  2.33s/it]

TPU test_1 not found


 67%|██████▋   | 4/6 [00:09<00:04,  2.32s/it]

TPU test_3 not found


100%|██████████| 6/6 [00:14<00:00,  2.34s/it]


In [ ]:
list_tpus()['nodes']

[{'name': 'projects/learning-from-play-303306/locations/us-central1-f/nodes/test_4',
  'acceleratorType': 'v2-8',
  'state': 'PREEMPTED',
  'runtimeVersion': 'v2-alpha',
  'cidrBlock': '10.128.0.0/20',
  'createTime': '2022-02-03T11:45:49.185763153Z',
  'schedulingConfig': {'preemptible': True},
  'networkEndpoints': [{'ipAddress': '10.128.0.24',
    'port': 8470,
    'accessConfig': {'externalIp': '34.72.165.227'}}],
  'id': '5283112417578115423',
  'networkConfig': {'network': 'projects/learning-from-play-303306/global/networks/default',
   'subnetwork': 'projects/learning-from-play-303306/regions/us-central1/subnetworks/default',
   'enableExternalIps': True},
  'serviceAccount': {'email': '940527310713-compute@developer.gserviceaccount.com',
   'scope': ['https://www.googleapis.com/auth/devstorage.read_write',
    'https://www.googleapis.com/auth/logging.write',
    'https://www.googleapis.com/auth/service.management',
    'https://www.googleapis.com/auth/servicecontrol',
    'http

In [ ]:
[make_pane(name) for name in names],

([({'test_0': ["printf '\x1b]2;%s\x1b\\' 'test_0'"]},),
  ({'test_1': ["printf '\x1b]2;%s\x1b\\' 'test_1'"]},),
  ({'test_2': ["printf '\x1b]2;%s\x1b\\' 'test_2'"]},),
  ({'test_3': ["printf '\x1b]2;%s\x1b\\' 'test_3'"]},)],)

In [ ]:
[{'a':[1,2]}, {'a':'b'}],

([{'a': [1, 2]}, {'a': 'b'}],)

In [ ]:
f"printf '\033]2;%s\033\\' '{name}'"

"printf '\x1b]2;%s\x1b\\' '5'"

In [ ]:
"rty"

'rty'